In [2]:
import cv2
import pytesseract
import subprocess

In [3]:
LANDMARKS = ["3", "2", "1"]

CROPS = {
    "countdown-crop": {
        "CROP_X": 800,
        "CROP_Y": 200,
        "CROP_HEIGHT": 200,
        "CROP_WIDTH": 200
    },
    "scoreboard-crop": {
        "CROP_X": 800,
        "CROP_Y": 200,
        "CROP_HEIGHT": 200,
        "CROP_WIDTH": 200
    },
    "minimap_crop": {
        "CROP_X": 800,
        "CROP_Y": 200,
        "CROP_HEIGHT": 200,
        "CROP_WIDTH": 200
    }
}
CROP_X, CROP_Y, CROP_WIDTH, CROP_HEIGHT = 800, 50, 200, 100

In [ ]:
def extract_countdown(clip_path):
    cap = cv2.VideoCapture(clip_path)
    timestamps = []
    fps = cap.get(cv2.CAP_PROP_FPS)  # noqa: F841

    while True:

        ret, frame = cap.read()

        if not ret:
            break

        text = pytesseract.image_to_string(frame, config="--psm 6")
        if text.strip() in LANDMARKS:
            timestamps.append(cap.get(cv2.CAP_PROP_POS_MSEC) / 1000)

    cap.release()
    return timestamps

In [5]:
def sync_videos(player_timestamps, caster_timestamps, video_paths):
    ref_time = min(caster_timestamps)

    for i, times in enumerate(player_timestamps):
        offset = min(times) - ref_time
        if offset > 0:

            output_path = f"synced_player{i+1}.mp4"
            cmd = f"ffmpeg -i video_paths[i] -ss {offset} -c copy {output_path}"
            subprocess.run(cmd, shell=True)
            print(f"Synced {video_paths[i]} to {output_path}")